In [1]:
# ✅ 통합 최종 코드 (정렬 포함)
# 1. 파일 경로
# file_path = r"D:\pythondata\4_회수&배송_방문코드_중복_전체데이터.csv"
# 칼럼은 14개(No,자전거번호,대여소,대여소명,날짜,시간,날짜+시간,방문코드,순번,flag,자전거대수,first data&time,last data&time,구분)로 구성되어 있어요.

# 파일을 보면 알겠지만, 회수와 배송은 "구분"칼럼에서 따로 적어뒀어요.
# 1. 이 파일에서 No는 정렬 순서(단순 정렬 개념)에 따라서 제일 윗줄부터 1로 시작하면 좋겠어요.
# 2. "대여소"(102번 대여소가 가장 위에 와야 함)별로 "자전거번호"가 "날짜+시간"의 30분이 넘어가면 "방문코드"가 변경되는 것을 원칙해야 해요. 
#    그리고 시작 번호는 "A0000001"로 시작했으면 좋겠어요. "순번"은 1부터인거 아시죠?
# 3. "flag" 칼럼은 동일한 방문코드일 때, 자전거번호가 30분 이내에 위아래 행에 위치하면 "가짜"라고 적으면 돼요.
# 4. 그리고 현재 파일(4_회수&배송_방문코드_중복_전체데이터.csv)에는 "first data&time"과 "last data&time"이 모두 채워져 있는데, 
#    동일한 방문코드의 순번이 1번이 "first data&time"에 위치하면 되고, 방문코드의 마지막 순번이 "last data&time"에 위치하면 돼요. 
#    그런데 "flag" 칼럼의 "가짜"라는 단어때문에 코딩이 꼬여서 모든 칸이 채워졌는데요, 해당 순번이 있는 행(row)에만 위치하면 돼요. 
#    "flag" 칼럼에 "가짜"라는 단어가 있어도, 동일 "방문코드"의 마지막 "순번"에서 "날짜+시간" 중 마지막이요!!

In [2]:
import pandas as pd

# 1. 파일 경로
file_path = r"D:\pythondata\4_회수&배송_방문코드_중복_전체데이터.csv"

# 2. 데이터 불러오기
# df = pd.read_csv(file_path, encoding="utf-8-sig", dtype={"대여소": str}) 
# -------------- DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
# C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_666380\2039930066.py:7: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
df = pd.read_csv(
    file_path,
    encoding="utf-8-sig",
    dtype={
        "대여소": str,
        "대여소명": str  # 문자열로 강제 지정
    }
)


# 3. 날짜+시간 변환
df["날짜+시간"] = pd.to_datetime(df["날짜+시간"])

# 4. 정렬: 대여소(숫자 변환, 102가 가장 위로), 자전거번호, 날짜+시간
df["대여소_정렬"] = pd.to_numeric(df["대여소"], errors="coerce")
df = df.sort_values(by=["대여소_정렬", "자전거번호", "날짜+시간"]).reset_index(drop=True)

# 5. 방문코드 생성: 30분 초과 또는 대여소/자전거번호 변경 시 새 코드
visit_codes = []
visit_index = 1
visit_codes.append(f"A{visit_index:07d}")  # A0000001

for i in range(1, len(df)):
    prev = df.loc[i - 1]
    curr = df.loc[i]
    diff_sec = (curr["날짜+시간"] - prev["날짜+시간"]).total_seconds()
    changed = (
        curr["대여소"] != prev["대여소"] or
        curr["자전거번호"] != prev["자전거번호"] or
        diff_sec > 1800
    )
    if changed:
        visit_index += 1
    visit_codes.append(f"A{visit_index:07d}")

df["방문코드"] = visit_codes

# 6. 순번 부여 (방문코드 그룹 내에서 1부터)
df["순번"] = df.groupby("방문코드").cumcount() + 1

# 7. flag: 동일 방문코드 내 자전거번호 중복이면 "가짜"
df["flag"] = ""
dup_mask = df.duplicated(subset=["방문코드", "자전거번호"], keep="first")
df.loc[dup_mask, "flag"] = "가짜"

# 8. 자전거대수 계산
df["자전거대수"] = df.groupby("방문코드")["자전거번호"].transform("count")

# 9. first/last data&time 설정: 순번 1과 마지막 순번인 경우만
df["first data&time"] = pd.NaT
df["last data&time"] = pd.NaT

first_mask = df["순번"] == 1
last_mask = df["순번"] == df.groupby("방문코드")["순번"].transform("max")

df.loc[first_mask, "first data&time"] = df["날짜+시간"]
df.loc[last_mask, "last data&time"] = df["날짜+시간"]

# 10. No 부여: 전체 정렬 후 위에서부터 1부터
df = df.sort_values(by=["대여소_정렬", "날짜+시간"]).reset_index(drop=True)
df["No"] = range(1, len(df) + 1)

# 11. 컬럼 순서 재정렬
col_order = [
    "No", "자전거번호", "대여소", "대여소명", "날짜", "시간", "날짜+시간",
    "방문코드", "순번", "flag", "자전거대수",
    "first data&time", "last data&time", "구분"
]
df = df[col_order]

# 12. 저장 (선택적)
df.to_csv(r"D:\pythondata\5_최종_회수배송_처리완료.csv", index=False, encoding="utf-8-sig")

print("✅ 처리 완료!")

✅ 처리 완료!
